## Jyve P5
[p5js](https://p5js.org/) is the spiritual successor to [`processing`](http://processingjs.org/), but uses bog-standard JavaScript for its execution... but does do some strange things to the environment to support its primary usage scenario.

Here's an adaptation of their [Game of Life Example](https://p5js.org/examples/simulate-game-of-life.html). That example uses [Global Mode](https://github.com/processing/p5.js/wiki/Global-and-instance-mode) which does all kind of weird things to the global scope. It has been changed to use Instance Mode on the object `s` (`sketch` is used in the examples), which seems to work much better with the Jyve model.

In [ ]:
s = {}

[object Object]

Fill board randomly

In [ ]:
function setup() {
  s.createCanvas(720, 800);
  s.w = 20;
  // Calculate columns and rows
  s.columns = s.floor(s.width/s.w);
  s.rows = s.floor(s.height/s.w);
  // Wacky way to make a 2D array is JS
  s.board = new Array(s.columns);
  for (var i = 0; i < s.columns; i++) {
    s.board[i] = new Array(s.rows);
  } 
  // Going to use multiple 2D arrays and swap them
  s.next = new Array(s.columns);
  for (i = 0; i < s.columns; i++) {
    s.next[i] = new Array(s.rows);
  }
  s.init();
}

In [ ]:
function init() {
  for (var i = 0; i < s.columns; i++) {
    for (var j = 0; j < s.rows; j++) {
      // Lining the edges with 0s
      if (i == 0 || j == 0 || i == s.columns-1 || j == s.rows-1) s.board[i][j] = 0;
      // Filling the rest randomly
      else s.board[i][j] = s.floor(s.random(2));
      s.next[i][j] = 0;
    }
  }
}

The process of creating the new generation

In [ ]:
function generate() {
  // Loop through every spot in our 2D array and check spots neighbors
  for (var x = 1; x < s.columns - 1; x++) {
    for (var y = 1; y < s.rows - 1; y++) {
      // Add up all the states in a 3x3 surrounding grid
      var neighbors = 0;
      for (var i = -1; i <= 1; i++) {
        for (var j = -1; j <= 1; j++) {
          neighbors += s.board[x+i][y+j];
        }
      }

      // A little trick to subtract the current cell's state since
      // we added it in the above loop
      neighbors -= s.board[x][y];
      // Rules of Life
      if      ((s.board[x][y] == 1) && (neighbors <  2)) s.next[x][y] = 0;             // Loneliness
      else if ((s.board[x][y] == 1) && (neighbors >  3)) s.next[x][y] = 0;             // Overpopulation
      else if ((s.board[x][y] == 0) && (neighbors == 3)) s.next[x][y] = 1;             // Reproduction
      else                                               s.next[x][y] = s.board[x][y]; // Stasis
    }
  }

  // Swap!
  var temp = s.board;
  s.board = s.next;
  s.next = temp;
}

In [ ]:
function draw() {
  s.background(0);
  s.generate();
  for ( var i = 0; i < s.columns;i++) {
    for ( var j = 0; j < s.rows;j++) {
      if ((s.board[i][j] == 1)) s.fill(0);
      else s.fill(255); 
      s.stroke(0);
      s.rect(i*s.w, j*s.w, s.w-1, s.w-1);
    }
  }

}

reset board when mouse is pressed

In [ ]:
function mousePressed() {
  s.init();
}

## A Hack
Apparently nothing happens until we instantiate `p5`. Also, as we are expecting things to run over and over again, when you get to this cell (i.e. with restart and run all cells), it tries to remove the sketch if it has already been installed.

Further, since the "real" `s` in question is a `p5` instance, we need to take some care to make sure everything is called correctly, even after it is changed interactively, hence all the wrappers. Similarly, one could define them initially as `s.setup = function(){...}`, but this is a little closer to the original example.

In [ ]:
s.remove && s.remove()
new p5((_s) => {
    s = _s;
    s.setup = () => setup();
    s.draw = () => draw();
    s.init = () => init();
    s.generate = () => generate();
    s.mousePressed = () => mousePressed();
    
})

[object Object]